In [1]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os, sys
from keras.models import Model
import csv
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

def load_image(root_path, height, width, classes):
    label_map = {'BacterialPneumonia': 1,
                 'COVID-19': 0,
                 'Normal': 2,
                 'ViralPneumonia': 1}
    data, labels = [], []
    dirs = os.listdir(root_path)
    if '.DS_Store' in dirs:
        dirs.remove('.DS_Store')
    for d in dirs:
        path = os.path.join(root_path, d)
        img_count = os.listdir(path)
        for img in img_count:
            try: 
                img_path = os.path.join(path, img)
                image = cv2.imread(img_path)
                image_from_array = Image.fromarray(image, 'RGB')
                size_image = image_from_array.resize((width, height))
                data.append(np.array(size_image))
                labels.append(label_map[d])
            except AttributeError:
                print(" ")
    
    Cells = np.array(data)
    labels = np.array(labels)
    
    s = np.arange(Cells.shape[0])
    np.random.seed(classes)
    np.random.shuffle(s)
    Cells = Cells[s]
    labels = labels[s]
    
    X = Cells.astype('float32')/255
    y = to_categorical(labels, classes)
    return X, y


def data_processing(data_path, height, width, classes):
    train_path = os.path.join(data_path, 'NonAugmentedTrain')
    val_path = os.path.join(data_path, 'ValData')
    X_train, y_train = load_image(train_path, height, width, classes)
    X_val, y_val = load_image(val_path, height, width, classes)
    return X_train, y_train, X_val, y_val




Using TensorFlow backend.


In [3]:
from __future__ import print_function
import tensorflow as tf
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm


def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=bk.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops


def piecewise5(X):
    return bk.switch(X < -0.6, (0.01 * X ),
                     bk.switch(X < -0.2, (0.2 * X ),
                               bk.switch(X < 0.2, (1 * X ),
                                         bk.switch(X < 0.6, (1.5 * X ),
                                                   bk.switch(X < 5, (3 * X ), (3 * X )))))) 


class CustomNetwork(nn.Module):
    def __init__(self, classes):
        super().__init__()
        self.conv_1 = nn.Conv2d(1, 16, kernel_size=1, padding=0)
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        self.avgpool_1 = nn.AdaptiveAvgPool2d((16, 16))
        self.maxpool_1 = nn.MaxPool2d(kernel_size=2, padding=0)
        self.dropout_1 = nn.Dropout(0.1)
        
        self.conv_3 = nn.Conv2d(32, 32, kernel_size=1, padding=0)
        self.conv_4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        
        self.maxpool_2 = nn.MaxPool2d(kernel_size=2, padding=0)
        self.dropout_2 = nn.Dropout(0.1)
        
        self.avgpool_2 = nn.AvgPool2d(kernel_size=4, padding=0)
        self.dropout_3 = nn.Dropout(0.1)
        
        self.avgpool_3 = nn.AvgPool2d(kernel_size=2, padding=0)
        self.dropout_4 = nn.Dropout(0.1)
        
        self.fc = nn.Linear(12304, classes)
        
    def forward(self, input_img):
        tower_1 = F.elu(self.conv_1(input_img))
        tower_x = F.elu(self.conv_2(tower_1))
        block1_output = self.avgpool_1(tower_1)
        
        tower_y = self.maxpool_1(tower_x)
        tower_y = self.dropout_1(tower_y)
        tower_z = F.elu(self.conv_3(tower_y))
        tower_a = F.elu(self.conv_4(tower_z))
        tower_a = self.maxpool_2(tower_a)
        tower_a = self.dropout_2(tower_a)
        
        tower_2 = self.avgpool_2(tower_x)
        tower_2 = self.dropout_3(tower_2)
        
        tower_3 = self.avgpool_3(tower_z)
        tower_3 = self.dropout_4(tower_3)
        
        output = torch.cat((tower_a, tower_2, tower_3), 1)
        output = torch.flatten(output)

        out1 = torch.cat((output, block1_output), 1)
        
        out = F.softmax(self.fc(out1), dim=1)
        return out

In [4]:
if __name__ == '__main__':
    height = 400
    width = 300
    channels = 3
    classes = 4
    ratio_train = 0.8
    ratio_val = 0.2
    save_path = './training_info.csv'
    pre_trained_model_path = ''
    model_save_path = './test2_model.h5'
    data_path = './covid19-detection-xray-dataset'
    
    if ratio_train + ratio_val > 1:
        print('Train/eval splitting failed')
        exit(0)
    X_train, y_train, X_val, y_val = data_processing(data_path,
                                                     height, 
                                                     width,  
                                                     classes)

    network = CustomNetwork(classes)    
    
    optimizer = optim.SGD(network.parameters(), lr=0.001)
    loss_function = nn.CrossEntropyLoss()
    input_shape = X_train.shape[1:]

    X_train_t = torch.Tensor(X_train)
    y_train_t = torch.Tensor(y_train)
    X_val_t = torch.Tensor(X_val)
    y_val_t = torch.Tensor(y_val)

    epochs = 5
    batch_size = 32
    for epoch in range(epochs):
        print(input_shape)
        for i in range(0, batch_size): 
            
            batch_X = X_train_t[i:i + batch_size].view(-1, 1, height, width)
            batch_y = y_train_t[i:i + batch_size]
        
            network.zero_grad()
        
            outputs = network.forward(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()   

    print(f"Epoch: {epoch}. Loss: {loss}")
        
    # get_custom_objects().update({'piecewise5': Activation(piecewise5)})
    # input_shape = X_train.shape[1:]

    # sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
    # model = custom_network(height, width, classes, pre_trained_model_path)
    
    # model.compile(loss=keras.losses.categorical_crossentropy,
    #               optimizer='adam',
    #               metrics=['accuracy'])
    
    # epochs = 5
    # hist1 = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))
    # model.save(model_save_path)  # should end with .h5 or .hdf5
    
    # history = hist1.history
    
    # score = model.evaluate(X_val, y_val, verbose=0)
    # score2 = model.evaluate(X_train, y_train, verbose=0)
    # print('Test loss:', score[0])
    # print('Test accuracy:', score[1])
    # print('Training loss:', score2[0])
    # print('Training accuracy:', score2[1])



(400, 300, 3)


RuntimeError: size mismatch, m1: [1 x 69120000], m2: [12304 x 4] at ..\aten\src\TH/generic/THTensorMath.cpp:41